In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6e92644df2897a8b54602d86886129191a7e65c6c51399ef6ba8a3cc79e37981
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession \
         .builder \
         .appName("PySpark Example") \
         .config("spark.some.config.option", "some-value")\
         .getOrCreate())

In [ ]:
spark

In [14]:
accounts = spark.read.option('header', 'true').csv('/content/accounts.csv')

In [15]:
accounts.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)



In [20]:
transactions = spark.read.option('header', True).parquet('./transactions.parquet')

In [21]:
transactions.count()

1000000

In [22]:
#New dataframe
account_transactions = transactions.groupby('account_number').sum()

In [23]:
with_sum = accounts.join(account_transactions, 'account_number', 'inner')

In [24]:
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))

In [25]:
neg_balance = accounts.filter(accounts.new_balance < 0)

In [27]:
clients = spark.read.json('./clients.json')

In [28]:
clients = clients.join(neg_balance, 'account_number', 'inner')

In [29]:
clients.select(['first_name', 'last_name', 'account_number', 'new_balance']).show(5)

+----------+---------+------------------+-----------+
|first_name|last_name|    account_number|new_balance|
+----------+---------+------------------+-----------+
|    Meagan| Sandoval|JMTP45763117901514|   -27573.0|
|  Michelle|   Knight|RBUE05237750254383|  -103459.0|
|      Paul|   Massey|RJMY57096756148587|   -58329.0|
|  Michelle|    Perez|ZYMB62177146259441|   -55431.0|
|     David|    Green|LRTH65732611614073|  -103831.0|
+----------+---------+------------------+-----------+
only showing top 5 rows

